In [48]:
import pandas as pd
import statsmodels.api as sm

In [52]:
SP = pd.read_csv('MSCI.csv', parse_dates=True, index_col='Date',)
SP

,GN,SP 500,MSCI World
Date,,,
2023-03-28,"140,7","3971,27","2,704.76"
2023-03-27,"143,4","3977,53","2,702.42"
2023-03-24,"146,7","3970,99","2,692.55"
2023-03-23,"145,2","3948,72","2,696.85"
2023-03-22,"143,6","3936,97","2,686.13"
...,...,...,...
NaT,NaN,NaN,"2,134.51"
NaT,NaN,NaN,"2,144.60"
NaT,NaN,NaN,"2,154.26"


In [53]:
SP = SP[SP.columns[SP.columns.isin(['Date','GN', 'MSCI World'])]]
SP['GN'] = SP['GN'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)
SP['MSCI World'] = SP['MSCI World'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)


/tmp/ipykernel_4168/3424614393.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP['GN'] = SP['GN'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)
/tmp/ipykernel_4168/3424614393.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP['MSCI World'] = SP['MSCI World'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)


In [54]:
SP.sort_values(by='Date', inplace = True) 
SP

/tmp/ipykernel_4168/1494837875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP.sort_values(by='Date', inplace = True)


,GN,MSCI World
Date,,
2018-03-08,218.3,2.11773
2018-03-09,220.2,2.11080
2018-03-12,222.6,2.10714
2018-03-13,222.0,2.08327
2018-03-14,221.8,2.10042
...,...,...
NaT,NaN,2.13451
NaT,NaN,2.14460
NaT,NaN,2.15426


In [55]:
daily_ret = SP.pct_change(1)
daily_ret_clean = daily_ret.dropna(axis=0)
daily_ret_clean

,GN,MSCI World
Date,,
2018-03-09,0.008704,-0.003272
2018-03-12,0.010899,-0.001734
2018-03-13,-0.002695,-0.011328
2018-03-14,-0.000901,0.008232
2018-03-15,0.005861,-0.003571
...,...,...
NaT,0.000000,0.000544
NaT,0.000000,0.004727
NaT,0.000000,0.004504


In [57]:

# split dependent and independent variable
X = daily_ret_clean['MSCI World']
y = daily_ret_clean['GN']

# Add a constant to the independent value
X1 = sm.add_constant(X)

# make regression model 
model = sm.OLS(y, X1)

# fit model and print results
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     GN   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.1426
Date:                Wed, 12 Apr 2023   Prob (F-statistic):              0.706
Time:                        12:43:51   Log-Likelihood:                 2960.7
No. Observations:                1318   AIC:                            -5917.
Df Residuals:                    1316   BIC:                            -5907.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6.828e-06      0.001     -0.010      0.9

In [58]:
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(X, y)

print(f'Beta = {slope}')

Beta = 0.020801572346123506
